In [ ]:
# Prepare environment
import os, sys
from pathlib import Path
PROJECT_ROOT_KEY = "PROJECT_ROOT"
PROJECT_ROOT = os.path.abspath(os.environ.get(PROJECT_ROOT_KEY, ".."))
if PROJECT_ROOT_KEY not in os.environ:
    os.environ[PROJECT_ROOT_KEY] = PROJECT_ROOT # Set 
sys.path.append(os.environ[PROJECT_ROOT_KEY])
for submodule in Path(PROJECT_ROOT).joinpath("submodules").glob("*"):
    sys.path.append(str(submodule))

# Environment
from src import *
os.environ["NUM_DEVICES"] = "1"
os.environ["HYDRA_FULL_ERROR"] = "1"

import open3d
# open3d.visualization.webrtc_server.disable_http_handshake()

# Other imports
import torch
import hydra
import numpy as np
import matplotlib.pyplot as plt
from hydra import initialize, compose

In [6]:
from src.utils.hydra import resolve_and_compose_model_processes
with initialize(version_base="1.3", config_path="../configs", job_name="dev"):
    cfg = compose(config_name="summary", overrides=[
    "inputs.regex=PhysNet3DCNN_PURE_rPPGToolbox_01dfb69df2a648fe954a4cf7b0dc41cd.HDF5"
    ])
    # cfg = resolve_and_compose_model_processes(cfg)

In [ ]:
import h5py
file = "PhysNet3DCNN_PURE_5be6c3561e664d3c85b0b207ba883c13.HDF5"
with h5py.File(file, "r") as fp:
    for index, vals in fp.items():
        for key, val in vals.items():
            print(index, key, val.keys())
        break

In [8]:
# files = hydra.utils.instantiate(cfg.inputs)
# process = hydra.utils.instantiate(cfg.processes)
# results = process(input=files)

In [9]:
def calc_snr(peak_hz, f_signal, pxx_signal, harmonic_delta, wn, db):
    # print(peak_hz, f_signal, pxx_signal)
    harmonic_1 = peak_hz
    harmonic_2 = 2 * harmonic_1

    # mask frequencies
    mask_harmonic_1 = (f_signal >= (harmonic_1 - harmonic_delta)) & (f_signal <= (harmonic_1 + harmonic_delta))
    mask_harmonic_2 = (f_signal >= (harmonic_2 - harmonic_delta)) & (f_signal <= (harmonic_2 + harmonic_delta))
    mask_remainder = (f_signal >= wn[0]) & (f_signal <= wn[1]) & ~mask_harmonic_1 & ~mask_harmonic_2
    pwr_harmonic_1 = np.sum(pxx_signal[mask_harmonic_1]) # across all dims
    pwr_harmonic_2 = np.sum(pxx_signal[mask_harmonic_2])
    pwr_remainder = np.sum(pxx_signal[mask_remainder])

    # calc power in bands
    if pwr_remainder != .0:
        snr = (pwr_harmonic_1 + pwr_harmonic_2) / pwr_remainder
        if db: snr = 20 * np.log10(snr)
    else:
        snr = (pwr_harmonic_1 + pwr_harmonic_2) * pwr_remainder
    
    return snr
 

In [10]:
# PURE rPPG-Toolbox Intra-dataset Testing
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_90042b2faa484547b5a6e1e4ad74209c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c2153b83e6ea430dbab61b26bd7484a4.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_08b40968bb8848aabc4fbafa96980557.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_e58de5f744614983a0e3a4c6f9af8149.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_405fd913761844459117445a6945144a.HDF5"
# ]

# # PURE (our process aka re-sampling + dynamic cropping) - intra-dataset testing
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_09c966b2b0c847e09fed8619978c7a2f.HDF5", # fold-1
#     "PROCESSED_PhysNet3DCNN_PURE_956a594a24fe43f985be7616d1536e60.HDF5", # fold-2
#     "PROCESSED_PhysNet3DCNN_PURE_1b949bb35074434483bce2ea15c32678.HDF5", # fold-3
#     "PROCESSED_PhysNet3DCNN_PURE_78fd5eb665444d5c9210e546585d60f9.HDF5", # fold-4
#     "PROCESSED_PhysNet3DCNN_PURE_c8f38e7fe03c4c97ac50970e811af585.HDF5" # fold-5
# ]

# # PURE : our replication + BVP re-sampling only
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_73e7d789f3c445a7bfc4e2d1fea29669.HDF5"
# ]

# # PURE : ablation 3 (square padded cropping)
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_5be6c3561e664d3c85b0b207ba883c13.HDF5" # sq-padded crop WITH re-sampling
# ]

# # PURE (default rppg-toolbox replication) : cross-dataset on MMPD
# files = [
#     "PROCESSED_PhysNet3DCNN_MMPD_90042b2faa484547b5a6e1e4ad74209c.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_405fd913761844459117445a6945144a.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_c2153b83e6ea430dbab61b26bd7484a4.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_08b40968bb8848aabc4fbafa96980557.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_e58de5f744614983a0e3a4c6f9af8149.HDF5"
# ]

# # PURE Ablation 1 (bvp re-sampling) - intra-dataset on PURE
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_99b8587c68364b479281ea2e2d876dfd.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_a47e049e8f8a4e11b0e391675bbe5395.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_3d05d869345142bc8ee5cf22ae59a51b.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_4050b78c9192420eb1970cb9712f8a50.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_fe8b3a3ded204b80bdaab1c98d586ec9.HDF5"
# ]

# # PURE Ablation 2 (dynamic cropping) - intra-dataset on PURE
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_8d6df0bdb650487dbf9de5d0e4d779e4.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_b2f74d6c9901485684ae3d0f694bdd82.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_adaa73dffcf143a298df74a56f5a2ac0.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_fb87dea3df4f49fd8c60c3f7d6cd785b.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c4976baf4bc24418a82670cc131bd5df.HDF5"
# ]

# PURE Ablation 3 (squared dynamic cropping) - intra-dataset on PURE
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_585c1b88a04e4879a5fc001f902a3e5f.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5f214091c4a7489eb87387af7deb7f57.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_2b36928e36ee480dae52324ae7bea196.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_4b5407c4c12d4d7e8e7ff46c821151f6.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_aa86c56a8fbe4ed8a3fc805645117fc7.HDF5"
# ]

# # # PURE Ablation 4 (scaled square dynamic cropping) - intra-dataset on PURE
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_ffe26bbc6e46464fa03b09bd020b95aa.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_b1637ab25003440aa69da3811189afab.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_f249fc4a196f4d47b632e484423083b1.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c377aa57dfff4565be5ee7e35eb8061f.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_468428e45db8425da05b9f1bda8ce2a6.HDF5"
# ]

# # PURE : Abation (normalized appearance frames) - intra-dataset on PURE
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_cc6815e557ea4fdcb32a668cd910752e.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5d77a5b67a3b47c0bcd83eb4392e8c26.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5a2954c1bda240c89c4d8d1fda674f7b.HDF5", # POOR TRAINING
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c529e666e9ff4bd9ad17f69f4de4001c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c3f1dd2cbfc6406990164342f42d3bb6.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_37149d9cbc3044e387f4e04bbff6643b.HDF5"
# ]

# PURE abaltion (facial segmentation) - intra-dataset on PURE
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_0386e197cd3e4e99bc8b683fcc5290ac.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_fb9368d0c7bf422ca06bf4ca230082f4.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_9a8bfa9d71a44d6aa49a7c2a055c3682.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_1a65c3fe5faa4afe8b935d91eef3928f.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_0905126275e84ce1b4c4de45c3163905.HDF5" 
# ]

# # PURE UV Coordinates - intra-dataste on PURE
files = [
    "PROCESSED_PhysNet3DCNN_PURE_UV_0333b0b95df24105b6491d5036d44a07.HDF5",
    "PROCESSED_PhysNet3DCNN_PURE_UV_721beb6e4a1f4ec2b8d91b1aff1cca84.HDF5",
    "PROCESSED_PhysNet3DCNN_PURE_UV_172f086783504abe85d55add32af1fdd.HDF5",
    "PROCESSED_PhysNet3DCNN_PURE_UV_61093ec89706473981dbdc5582d461e8.HDF5",
    "PROCESSED_PhysNet3DCNN_PURE_UV_e92e412d2fca4f5186341ec58d30df5c.HDF5"
]

# # MMPD UV Coords - cross-dataset testing onto MMPD
# files = [
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_0333b0b95df24105b6491d5036d44a07.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_721beb6e4a1f4ec2b8d91b1aff1cca84.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_172f086783504abe85d55add32af1fdd.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_61093ec89706473981dbdc5582d461e8.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_e92e412d2fca4f5186341ec58d30df5c.HDF5"
# ]

# # PURE frame-diff first
# files = [
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_cf24de22c7874ff783792c9717f78e5f.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_2c9be3db2c14417d911cef9173b7def4.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_deba75b2160f410f80d104a8e3a770d4.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_67027654175d4994ac47eaf23211a40c.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_3498efb5b7cc441b8008dff6a67aee8f.HDF5"
# ]

# PURE UNCROPPED : Intra-dataset Testing on PURE
# files = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_83e8a56aa71e48e3b6900b82955dc63c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_e0bd1672755d420fbc144548af0b7775.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_a0e37d0e43c24e5eb33b4ac7db0dba44.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_56e14c8eba944143a883481ad9d35e66.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_55bfe943bf3c416181f0f08719be1c23.HDF5"
# ]

# Experiments
06/03/2024

In [160]:
# # INTRA PURE: physnet_uncropped
# physnet_uncropped = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_490bcf666ed349ad9aff7fe1ca89d1b8.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_f9042fb12e0a494cac70d83942dd8979.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_1ce9df3f0aeb48a2af87faa9c53df145.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_37fb581b10c0434885bc1a1ab0a20612.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_948ec502f86642498b61d988969d7943.HDF5",
# ]
# physnet_uncropped_modfreq = [
#     "PROCESSED_MODFREQ_PhysNet3DCNN_PURE_rPPGToolbox_f9042fb12e0a494cac70d83942dd8979.HDF5"
# ]

# # INTRA PURE: physnet_rppg_toolbox_scale1
# physnet_rppg_toolbox_scale1 = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_49a280786518409191e4e75ddac9b219.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_bffe203b366a4df98d032decefdde6cc.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_d1fe5945d2a34f36b14659bc3236ffe0.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_fc138aaeac1f4e25b7232e8870859664.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_3a57dea2c436460fb77a43b951585cba.HDF5",
# ]

# # INTRA PURE: physnet_rppg_toolbox
# physnet_rppg_toolbox = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_4353aebc5a7a4e3cbf8fff24be490728.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_7deacfc0cbf04a5e8c4198c5b7930b27.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_39019eddc4dd45a4922fe97000dbdb80.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5c6e38afed244729bd69fdc845b0d602.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_041fbbcce1f04cb5b84e8b27defe60f7.HDF5",
# ]

# # INTRA PURE: physnet_rppg_toolbox_clipnorm
# physnet_rppg_toolbox_clipnorm = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_263ef7bf4fb04b79a6fc23ad6b0314cf.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_a4fbfd966df5449595f5ae0a51a42b9d.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_954e71769db948faab01032100aa6b89.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_a56f292f71c84b9ea7244388c5e42b65.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_fa54666e933e4c69aa955d12ef90caf1.HDF5",
# ]

# # INTRA PURE: physnet_rppg_toolbox_clipnorm_after
# physnet_rppg_toolbox_clipnorm_after = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_741d626a0fe0443b956f9ef01e7b7372.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_794853a1e9eb492cb7a3dcdf94b616ef.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_9fd4f9e133a741fc9c34bbe366bedc25.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_19a67866addc497baa0cddb9b5e85c1f.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c18321fd069048f1a46740e34750a003.HDF5",
# ]

# # INTRA PURE: physnet_rppg_toolbox_diffBef_stdAft
# physnet_rppg_toolbox_diffBef_stdAft = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_162e6f55e4ef4c039564eec91e46848c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_aeef6cdf21f34214a659a7ab7799f565.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_32a1c9f4ad5042059788b06515a1ed19.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_9a8777408dd4449b8a2f63b81655d327.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5b80d3ecc55742f896472d97d84c287c.HDF5",
# ]

# # INTRA PURE: physnet_dynamicCrop
# physnet_dynamicCrop = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c7b6652f4dc7421a8f04bed238d64448.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_9b49d263592147e78f2485554f2982d1.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_e55a08dbcc004a45a52ac87d8bcd8aec.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_96fc747266744417a0c7ac7e194f4a2d.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_23efa76f8d194631bf91e46b34ed2824.HDF5",
# ]

# # INTRA PURE: physnet_dynamicCrop_diffAfter
# physnet_dynamicCrop_diffAfter = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_9c807e16817d40b59335c152a054df68.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_ed4246a0cc034e74a5c0c1ef482dc174.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_2281244b91a7416b92cf69e423866b14.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5012c7b8c24e48edb5e1cba6b2dd3a60.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_778840eb62ed4c739428387020d82616.HDF5",
# ]

# # INTRA PURE: physnet_dynamicCrop_square
# physnet_dynamicCrop_square = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_f64c3e67762d476babe8b18bd85dc889.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c64738d706fe4d72ad930be9759140fd.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_530a7920f56e49bcbcf841b800a4eb9c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_f26a997a0a384a61bec193798dc3989c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_48d4f987d9124aa992fd68eeb69da6bb.HDF5",
# ]

# # INTRA PURE: physnet_scaleDynamicCrop_square
# physnet_scaleDynamicCrop_square = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_e66cee7b29fe484da1dbbc364f25be08.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_025eef4ec6e747d0ab1b2429be23388a.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_8c9ba2ee15da477fbe945f2d6797117b.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_8e6603ba647846fb9a6bda8641726cb9.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_467ac20f7b9441cc8825b37364eabda2.HDF5",
# ]

# # INTRA PURE: physnet_dynamicCropSegment
# physnet_dynamicCropSegment = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_519570afddaa494a93a6012461652ff6.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_2b1e5e38d41f4861bb18c2e86dee374d.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_707e3bee0ee944218ab41e62aa093903.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_e111c05da8f24313b051283060c11633.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_75bcce8fc45b4ca79996b0cd95e4b3d3.HDF5",
# ]

# # INTRA PURE: physnet_dynamicCropSegment_square
# physnet_dynamicCropSegment_square = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_46d98e337730480aab8160a9cf423627.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c1a1b1ede94e4c52b38b8d2382e3ccb5.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_9a219cf362e04a13b76f3ecd402c2a87.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_4f11d60a7f464170add6167b5c119bcc.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_115a183384f34f7689d4caa4bbb4578b.HDF5",
# ]

# # INTRA PURE: physnet_rppg_toolbox_rgb
# physnet_rppg_toolbox_rgb = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5f288fed06014524984c63fcee14a191.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5f47bc45697249c59ee021c4296c8a32.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_3380940cb02844d79c9f4f96c3d6dedb.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_9358f66dc7234ee9834fc5dc77744014.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_3c9160c235624986b2ae1b345b7b1a21.HDF5",
# ]

# # INTRA PURE: physnet_rppg_toolbox_rgb_std
# physnet_rppg_toolbox_rgb_std = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_c45e8251a43e47719eac65cc9c6b482a.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_238f800d32254868ad83e69cec9a86ba.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_546762f5c4784ca6bed0cd25727a4757.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_cf2ac260c12c49c5966b3dd5115ada9a.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_6e7bc99d587c44eba7c6e396f6d1fb41.HDF5",
# ]

# # INTRA PURE: physnet_rppg_toolbox_rgb_standardize
# physnet_rppg_toolbox_rgb_standardize = [
#     "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_f071ab6738b44504bc29e3082a4aa614.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_5e99f9a56f47415b928fbaadab972d9c.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_rPPGToolbox_.HDF5",
# ]


# # ----- [ UV ] -----

# # INTRA PURE: physnet_uv_all
# physnet_uv_all = [
#     "PROCESSED_PhysNet3DCNN_PURE_UV_c913138eb3b6406a9e5b891e458ab30a.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_cc595899e26b474e8193e879ce148993.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_55fb5fd66f89488a9b9d21d0ec643a06.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_609e988e229c47c5b640570009ca45cd.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_0c7dc44c6e29455295feab5a3972841a.HDF5",
# ]

# # INTRA PURE: physnet_uv_all
# physnet_uv_diff = [
#     "PROCESSED_PhysNet3DCNN_PURE_UV_37f018f8efdd4b14ae1b2a826644546b.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_f1dba09e67574756bd3a877ce0aaddd2.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_9d32f344169644eb8266f06d8d65536c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_63c649ced78d4dde9cde7343aa0a82e1.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_0c7dc44c6e29455295feab5a3972841a.HDF5",
# ]

# # INTRA PURE: physnet_uv_all
# physnet_uv_mask90 = [
#     "PROCESSED_PhysNet3DCNN_PURE_UV_2355b31af03d4b60bb5d8fada0f38583.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_49bf2afbe5bf45d9befa01493cb18e96.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_b713f9ab0a7d4955a54f993873620460.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_a7f071d0402a49b09d87c1c1cacdb7e9.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_2a8df4a83fbe425ba9ed9c355384192f.HDF5",
# ]

# # INTRA PURE: physnet_uv_all
# physnet_uv_mask60 = [
#     "PROCESSED_PhysNet3DCNN_PURE_UV_398198ec8ce74836b6c5303539f9e64c.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_bb7f25bcffa046da94638e1ac73b8ce5.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_85dc0bc804c6419e9aa694189dcadd31.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_66b941dcee714d9db30d69ce5a84d3d9.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_0aba21f9ce03459f948122c5724f422d.HDF5",
# ]

# # INTRA PURE: physnet_uv_all
# physnet_uv_mask45 = [
#     "PROCESSED_PhysNet3DCNN_PURE_UV_eaca70f7d3cb4042bbad465e80c70b4d.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_cb0c4aad82f44fc2a8738fb8da7b3afe.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_57ceced3f258481e8410fa1311281c6a.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_eb9cb38115f24359bf7e1d2b3e9edefd.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_9183c9307b4f448994c9a57355e56bae.HDF5",
# ]

# # INTRA PURE: physnet_uv_all
# physnet_uv_mask30 = [
#     "PROCESSED_PhysNet3DCNN_PURE_UV_dc75f2a8c9fa4bfbb891f6ac1f0d8344.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_4ce56affabce43c88f16745f8a09bd76.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_133574ffa77644baa05af60ccb81825f.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_e96827bb0cfa4e28816b674a51bc65af.HDF5",
#     "PROCESSED_PhysNet3DCNN_PURE_UV_d087d99feb4744cbb739417794661a5f.HDF5",
# ]

# # INTRA PURE: physnet_uv_mask90_rgb
# physnet_uv_mask90_rgb = [
#     "PROCESSED_PhysNet3DCNN_PURE_UV_a11ec8a362ef488db667b32963aad60e.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
# ]

# # INTRA PURE: physnet_uv_mask90_rgb
# physnet_uv_mask60_rgb = [
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
# ]

# # INTRA PURE: physnet_uv_mask90_rgb
# physnet_uv_mask45_rgb = [
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
# ]

# # INTRA PURE: physnet_uv_mask90_rgb
# physnet_uv_mask30_rgb = [
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_PURE_UV_.HDF5",
# ]

# file_opts = {
#     "physnet_uncropped_modfreq": physnet_uncropped_modfreq,
#     "physnet_rppg_toolbox_rgb": physnet_rppg_toolbox_rgb,
#     "physnet_rppg_toolbox_rgb_std": physnet_rppg_toolbox_rgb_std,
#     "physnet_rppg_toolbox_rgb_standardize": physnet_rppg_toolbox_rgb_standardize,
#     "physnet_uncropped": physnet_uncropped,
#     "physnet_rppg_toolbox_scale1": physnet_rppg_toolbox_scale1,
#     "physnet_rppg_toolbox": physnet_rppg_toolbox,
#     "physnet_rppg_toolbox_clipnorm": physnet_rppg_toolbox_clipnorm,
#     "physnet_rppg_toolbox_clipnorm_after": physnet_rppg_toolbox_clipnorm_after,
#     "physnet_rppg_toolbox_diffBef_stdAft": physnet_rppg_toolbox_diffBef_stdAft,
#     "physnet_dynamicCrop": physnet_dynamicCrop,
#     "physnet_dynamicCrop_diffAfter": physnet_dynamicCrop_diffAfter,
#     "physnet_dynamicCrop_square": physnet_dynamicCrop_square,
#     "physnet_scaleDynamicCrop_square": physnet_scaleDynamicCrop_square,
#     "physnet_dynamicCropSegment": physnet_dynamicCropSegment,
#     "physnet_dynamicCropSegment_square": physnet_dynamicCropSegment_square,
#     "physnet_uv_all": physnet_uv_all,
#     "physnet_uv_diff": physnet_uv_diff,
#     "physnet_uv_mask90": physnet_uv_mask90,
#     "physnet_uv_mask60": physnet_uv_mask60,
#     "physnet_uv_mask45": physnet_uv_mask45,
#     "physnet_uv_mask30": physnet_uv_mask30,
#     "physnet_uv_mask90_rgb": physnet_uv_mask90_rgb,
#     "physnet_uv_mask60_rgb": physnet_uv_mask60_rgb,
#     "physnet_uv_mask45_rgb": physnet_uv_mask45_rgb,
#     "physnet_uv_mask30_rgb": physnet_uv_mask30_rgb
# }

In [161]:
# # REFERENCE THE FILES TO USE HERE   
# key = "physnet_uv_mask45"
# files = file_opts[key]

In [169]:
# # CROSS PURE MMPD: physnet_rppg_toolbox
# physnet_rppg_toolbox_cross = [
#     # "PROCESSED_PhysNet3DCNN_MMPD_4353aebc5a7a4e3cbf8fff24be490728.HDF5",
#     # "PROCESSED_PhysNet3DCNN_MMPD_7deacfc0cbf04a5e8c4198c5b7930b27.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_39019eddc4dd45a4922fe97000dbdb80.HDF5",
#     # "PROCESSED_PhysNet3DCNN_MMPD_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_MMPD_.HDF5",
# ]

# # CROSS PURE MMPD: physnet_rppg_toolbox_clipnorm
# physnet_rppg_toolbox_clipnorm = [
#     "PROCESSED_PhysNet3DCNN_MMPD_263ef7bf4fb04b79a6fc23ad6b0314cf.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_a4fbfd966df5449595f5ae0a51a42b9d.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_954e71769db948faab01032100aa6b89.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_a56f292f71c84b9ea7244388c5e42b65.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_fa54666e933e4c69aa955d12ef90caf1.HDF5",
# ]

# # ----- [ UV ] -----

# # CROSS PURE MMPD: physnet_uv_all
# physnet_uv_all = [
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_c913138eb3b6406a9e5b891e458ab30a.HDF5",
#     # "PROCESSED_PhysNet3DCNN_MMPD_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_MMPD_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_MMPD_.HDF5",
#     # "PROCESSED_PhysNet3DCNN_MMPD_.HDF5",
# ]

# # CROSS PURE MMPD: physnet_uv_diff
# physnet_uv_diff = [
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_37f018f8efdd4b14ae1b2a826644546b.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_f1dba09e67574756bd3a877ce0aaddd2.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_9d32f344169644eb8266f06d8d65536c.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_63c649ced78d4dde9cde7343aa0a82e1.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_8d0bee41f30548d48faccc869f4c2465.HDF5",
# ]

# # CROSS PURE MMPD: physnet_rppg_toolbox_clipnorm
# physnet_uv_mask90 = [
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_2355b31af03d4b60bb5d8fada0f38583.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_49bf2afbe5bf45d9befa01493cb18e96.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_b713f9ab0a7d4955a54f993873620460.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_a7f071d0402a49b09d87c1c1cacdb7e9.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_2a8df4a83fbe425ba9ed9c355384192f.HDF5",
# ]

# # CROSS PURE MMPD: physnet_rppg_toolbox_clipnorm
# physnet_uv_mask60 = [
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_398198ec8ce74836b6c5303539f9e64c.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_bb7f25bcffa046da94638e1ac73b8ce5.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_85dc0bc804c6419e9aa694189dcadd31.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_66b941dcee714d9db30d69ce5a84d3d9.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_0aba21f9ce03459f948122c5724f422d.HDF5",
# ]

# # CROSS PURE MMPD: physnet_rppg_toolbox_clipnorm
# physnet_uv_mask45 = [
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_eaca70f7d3cb4042bbad465e80c70b4d.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_cb0c4aad82f44fc2a8738fb8da7b3afe.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_57ceced3f258481e8410fa1311281c6a.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_eb9cb38115f24359bf7e1d2b3e9edefd.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_9183c9307b4f448994c9a57355e56bae.HDF5",
# ]

# # CROSS PURE MMPD: physnet_rppg_toolbox_clipnorm
# physnet_uv_mask30 = [
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_dc75f2a8c9fa4bfbb891f6ac1f0d8344.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_4ce56affabce43c88f16745f8a09bd76.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_133574ffa77644baa05af60ccb81825f.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_e96827bb0cfa4e28816b674a51bc65af.HDF5",
#     "PROCESSED_PhysNet3DCNN_MMPD_UV_d087d99feb4744cbb739417794661a5f.HDF5",
# ]

# file_opts_cross = {
#     "physnet_rppg_toolbox_cross": physnet_rppg_toolbox_cross,
#     "physnet_rppg_toolbox_clipnorm": physnet_rppg_toolbox_clipnorm,
#     "physnet_uv_all": physnet_uv_all,
#     "physnet_uv_diff": physnet_uv_diff,
#     "physnet_uv_mask90": physnet_uv_mask90,
#     "physnet_uv_mask60": physnet_uv_mask60,
#     "physnet_uv_mask45": physnet_uv_mask45,
#     "physnet_uv_mask30": physnet_uv_mask30
# }

In [170]:
# # REFERENCE THE FILES TO USE HERE   
# key = "physnet_rppg_toolbox_clipnorm"
# files = file_opts_cross[key]

In [ ]:
import h5py

# Outputs
target_hr_bpms = []
prediction_hr_bpms = []
target_snrs = []
prediction_snrs = []

# Results relevant to a specific reporting
RELEVANT_RESULTS = []
errors = {}

count = 0

# 
for filepath in files:
    print(filepath)
    with h5py.File(filepath, "r") as fp:

        for idx, (name, vals) in enumerate(fp.items()):
            #
            count += 1
            # continue

            # Results
            attrs = vals["source/sample"].attrs
            t_bpm = vals["predictions/signal_processed"].attrs["peak_bpm"]
            p_bpm = vals["targets/labels_unnormalized_processed"].attrs["peak_bpm"]
            # print(list(attrs.values()))
            
            # Scenario
            subject = attrs["subject"]
            if "scenario" in attrs.keys():
                scenario = attrs["scenario"]
            elif "motion" in attrs.keys():
                scenario = attrs["motion"]
            else:
                raise ValueError

            # Errors for plotting
            errors[f"{subject}_{scenario}"] = abs(float(t_bpm) - float(p_bpm))

            # Conditionally accumulate results
            if scenario != "Stationary": continue

            # Heart-rate
            target_hr_bpms.append(float(t_bpm))
            prediction_hr_bpms.append(float(p_bpm))

            # SNR FFT
            prediction_snrs.append(calc_snr(
                vals["predictions/signal_processed"].attrs["peak_hz"],
                vals["predictions/signal_processed"].attrs["frequencies"],
                vals["predictions/signal_processed"].attrs["power"],
                0.1,
                [0.70, 2.50],
                True
            ))

print(f"Processed {count} samples.")

In [ ]:
target_hr_bpms = np.array(target_hr_bpms)
prediction_hr_bpms = np.array(prediction_hr_bpms)
target_snrs = np.array(target_snrs)
prediction_snrs = np.array(prediction_snrs)
n_samples = target_hr_bpms.shape[0]

print(key)

print(f" --- [ {key} ] ---")
mae = np.mean(np.abs(prediction_hr_bpms - target_hr_bpms))
mae_err = np.std(np.abs(prediction_hr_bpms - target_hr_bpms)) / np.sqrt(n_samples)
print(f"MAE: {mae:.5f} +/- {mae_err:.5f}")

rmse = np.sqrt(np.mean(np.square(prediction_hr_bpms - target_hr_bpms)))
rmse_err = np.std(np.square(prediction_hr_bpms - target_hr_bpms)) / np.sqrt(n_samples)
print(f"RMSE: {rmse:.5f} +/- {rmse_err:.5f}")

mape = np.mean(np.abs((prediction_hr_bpms - target_hr_bpms) / target_hr_bpms)) * 100
mape_err = np.std(np.abs((prediction_hr_bpms - target_hr_bpms) / target_hr_bpms)) / np.sqrt(n_samples)
print(f"MAPE: {mape:.5f} +/- {mape_err:.5f}")

pcor = np.corrcoef(prediction_hr_bpms, target_hr_bpms)[0][1]
pcor_err = np.sqrt((1 - pcor**2) / (n_samples - 2))
print(f"p CORR: {pcor:.5f} +/- {pcor_err:.5f}")
 
snr = np.mean(prediction_snrs)
snr_err = np.std(prediction_snrs) / np.sqrt(n_samples)
print(f"SNR: {snr:.5f} +/- {snr_err:.5f}")
print("")

In [ ]:
# Errors
fig, ax = plt.subplots(figsize=(16,4))
ax.bar(errors.keys(), errors.values())
ax.set_xticklabels(errors.keys(), rotation=90, ha="center")
ax.set_title(f"{key}")
ax.set_ylabel("HR Absolute Error (BPM)")
ax.grid(True, alpha=0.25)
fig.savefig(f"ERRORS_{key}.png")